In [1]:
import pandas as pd

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

print("Train DataFrame:")
print(train_df.describe())
print(train_df)


Train DataFrame:
            group_id
count  263859.000000
mean   545809.847525
std    249488.149296
min    101697.000000
25%    311997.000000
50%    524635.000000
75%    752266.000000
max    998545.000000
                                               image_link  group_id  \
0       https://m.media-amazon.com/images/I/61I9XdN6OF...    748919   
1       https://m.media-amazon.com/images/I/71gSRbyXmo...    916768   
2       https://m.media-amazon.com/images/I/61BZ4zrjZX...    459516   
3       https://m.media-amazon.com/images/I/612mrlqiI4...    459516   
4       https://m.media-amazon.com/images/I/617Tl40LOX...    731432   
...                                                   ...       ...   
263854  https://m.media-amazon.com/images/I/612J1R1xHl...    558806   
263855  https://m.media-amazon.com/images/I/61Blzh2+28...    470067   
263856  https://m.media-amazon.com/images/I/51MsegDL9V...    204245   
263857  https://m.media-amazon.com/images/I/510KhVw4VS...    752266   
263858  https

In [2]:
test_df


,index,image_link,group_id,entity_name
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth
...,...,...,...,...
131182,131283,https://m.media-amazon.com/images/I/A1rVsIzEtk...,721522,maximum_weight_recommendation
131183,131284,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,item_weight
131184,131285,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,maximum_weight_recommendation
131185,131286,https://m.media-amazon.com/images/I/A1tnTUPyr7...,853009,item_weight


In [3]:
num_columns = train_df.shape[1]
column_names = train_df.columns.tolist()
print(f"Total number of columns in train data: {num_columns}")
print("Column names:")
print(column_names)

Total number of columns in train data: 4
Column names:
['image_link', 'group_id', 'entity_name', 'entity_value']


In [4]:
train_50_df = train_df.head(50)


In [5]:
pip install paddlepaddle paddleocr opencv-python matplotlib Pillow pandas tqdm


In [6]:
import os
import time
import urllib.request
from pathlib import Path
from PIL import Image, ImageEnhance
import pandas as pd
import multiprocessing
from tqdm import tqdm
from functools import partial
from paddleocr import PaddleOCR

ocr = PaddleOCR(use_angle_cls=True, lang='en')



def preprocess_image(image_path):
    try:
        with Image.open(image_path) as img:
            # Convert image to grayscale to enhance text contrast
            img = img.convert('L')
            # Increase contrast
            enhancer = ImageEnhance.Contrast(img)
            img = enhancer.enhance(2.0)
            img.save(image_path)
    except Exception as e:
        print(f"Error preprocessing image {image_path}: {e}")

# Function to download a single image
def download_image(image_link, save_folder, retries=3, delay=3):
    if not isinstance(image_link, str):
        return

    filename = Path(image_link).name
    image_save_path = os.path.join(save_folder, filename)

    if os.path.exists(image_save_path):
        return image_save_path

    for _ in range(retries):
        try:
            urllib.request.urlretrieve(image_link, image_save_path)
            # Preprocess the image after downloading
            preprocess_image(image_save_path)
            return image_save_path
        except Exception as e:
            print(f"Error downloading {image_link}: {e}")
            time.sleep(delay)

    # If download fails, create a placeholder image
    create_placeholder_image(image_save_path)
    return image_save_path

# Function to download images from a list of URLs
def download_images(image_links, download_folder, allow_multiprocessing=True):
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    if allow_multiprocessing:
        download_image_partial = partial(download_image, save_folder=download_folder, retries=3, delay=3)

        with multiprocessing.Pool(64) as pool:
            image_paths = list(tqdm(pool.imap(download_image_partial, image_links), total=len(image_links)))
            pool.close()
            pool.join()
    else:
        image_paths = []
        for image_link in tqdm(image_links, total=len(image_links)):
            image_path = download_image(image_link, save_folder=download_folder, retries=3, delay=3)
            image_paths.append(image_path)

    return image_paths

# Function to perform OCR on downloaded images
def process_images_with_ocr(image_paths):
    ocr_results = []
    for image_path in image_paths:
        result = ocr.ocr(image_path, cls=True)
        text_output = []
        for idx in range(len(result)):
            res = result[idx]
            for line in res:
                text_output.append(line[1][0])  # Extract detected text
        ocr_results.append(' '.join(text_output))  # Join detected text in one string
        print(image_path)
        print(text_output)
    return ocr_results

# Assume this DataFrame contains image URLs


# Set the folder where images will be saved
download_folder = '/content/downloaded_images'

# Download images from URLs
image_links = train_50_df['image_link'].tolist()
image_paths = download_images(image_links, download_folder, allow_multiprocessing=True)

# Perform OCR on the downloaded images
ocr_results = process_images_with_ocr(image_paths)

# Add the OCR results to the DataFrame
train_50_df['ocr_text'] = ocr_results

# Show the DataFrame with extracted text
print(train_50_df[['image_link', 'ocr_text']])


[2024/09/16 11:15:07] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_c

100%|██████████| 50/50 [00:00<00:00, 217096.48it/s]


[2024/09/16 11:15:12] ppocr DEBUG: dt_boxes num : 14, elapsed : 1.0109708309173584
[2024/09/16 11:15:12] ppocr DEBUG: cls num  : 14, elapsed : 0.2945690155029297
[2024/09/16 11:15:18] ppocr DEBUG: rec_res num  : 14, elapsed : 5.814970016479492
/content/downloaded_images/61I9XdN6OFL.jpg
['PROPOS', 'NATURE', 'INGREDIENT MENAGER', 'MULTI-USAGE', 'TERREDE', 'SOMMIERES', '100%NATUREL', 'Argile 100% pure et naturelle, la terre de', 'Sommieres presente des proprietes absorbantes', 'recalcitrantes sur toutes les surfaces (moquette,', 'tapis parquet..Elle est aussi efficace pour', 'desodoriser le linge.', 'ngredientBentonite', '500g']
[2024/09/16 11:15:19] ppocr DEBUG: dt_boxes num : 12, elapsed : 0.5167405605316162
[2024/09/16 11:15:19] ppocr DEBUG: cls num  : 12, elapsed : 0.0909578800201416
[2024/09/16 11:15:22] ppocr DEBUG: rec_res num  : 12, elapsed : 3.175614833831787
/content/downloaded_images/71gSRbyXmoL.jpg
['TEARRIFIC', 'LEBENSMITTELECHT', 'M', 'GEPRAGTES', 'DESIGN', 'AS YOULIKE', 'BE

In [7]:
train_50_df['ocr_text'][3]

'therngiing 460trNS1202NS468id 1201ning  NS470 ::118 RDA CONTAINS PERMITTED SYNTHETIC F00D C0LOURS [INS 171.INS 172().INS 172&INS 172G] 0.51g S/ 700 n Punarnava Boerhavia diffusa root extract-0.07z alkaloids Bhui amla (Phyllanthus amarus) WP extract-0.5Z Bitters Kutki Picrorhiza kurroa rhizome extract-0.5z Bitters Dandelion Taraxacum officinale leaf extract-161 Kasani Cichorum endivia seed eract-1z Bitters Amla Phylanthus emblica) fruit extract-10%Tannins PHOSPHOcomplexSilybin Silybum msrianum) Piper aigrum fruit extract-95z Piperine SAXRSXAANOINE a medical conditien. COMPOSITION (sseains) Berey Protein SEDEON'

In [8]:
train_50_df.head()

,image_link,group_id,entity_name,entity_value,ocr_text
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram,PROPOS NATURE INGREDIENT MENAGER MULTI-USAGE T...
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup,TEARRIFIC LEBENSMITTELECHT M GEPRAGTES DESIGN ...
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram,COMPOSITION Serving Size:1 Tablet 0.709 g)Each...
3,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,item_weight,0.709 gram,therngiing 460trNS1202NS468id 1201ning NS470 ...
4,https://m.media-amazon.com/images/I/617Tl40LOX...,731432,item_weight,1400 milligram,Horbaach HIGHSTRENGTH PSYLLIOM HUSK 1400 PLANT...


In [9]:
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh # download ollama api
from IPython.display import clear_output

# Create a Python script to start the Ollama API server in a separate thread

import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
pciutils is already the newest version (1:3.7.0-6).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [10]:
from IPython.display import clear_output
!ollama pull llama3.1:8b
clear_output()

In [11]:
!pip install -U lightrag[ollama]

In [12]:
from lightrag.core.generator import Generator
from lightrag.core.component import Component
from lightrag.core.model_client import ModelClient
from lightrag.components.model_client import OllamaClient

qa_template = r"""<SYS>
You are a helpful assistant specialized in extracting specific information from text based on the given entity name. The following are the possible units for different entities:

- **width**: cm, centimetre, centimeter, m, metre, meter, mm, millimetre, millimeter, in, inch, ft, foot, yd, yard
- **depth**: cm, centimetre, centimeter, m, metre, meter, mm, millimetre, millimeter, in, inch, ft, foot, yd, yard
- **height**: cm, centimetre, centimeter, m, metre, meter, mm, millimetre, millimeter, in, inch, ft, foot, yd, yard
- **item_weight**: g, gram, gramme, kg, kilogram, kilogramme, mg, milligram, milligramme, oz, ounce, lb, pound, t, ton, tonne
- **maximum_weight_recommendation**: g, gram, gramme, kg, kilogram, kilogramme, mg, milligram, milligramme, oz, ounce, lb, pound, t, ton, tonne
- **voltage**: v, volt, kv, kilovolt, mv, millivolt
- **wattage**: w, watt, kw, kilowatt
- **item_volume**: ml, milliliter, millilitre, l, liter, litre, cl, centiliter, centilitre, fl oz, fluid ounce, gal, gallon, pt, pint, qt, quart, cup

From the text and entity name given to you, extract the numeric value and its corresponding unit in full form. Ensure that the unit is always reported in its singular form (e.g., '5 kg' should be returned as '5 kilogram'). If the value is a range (e.g., '240-260v'), return the higher number in the range. If no valid match is found, return an empty string.

Instructions:
1. Analyze the extracted text thoroughly for any mention of quantities and units related to the specified entity.
2. Identify the relevant value and unit based on the entity type. Always report the unit in singular form, regardless of the original text (e.g., '500 grams' should be reported as '500 gram').
3. If the value is given as a range (e.g., '240-260v'), select and return the higher number from the range.
4. Return the result in the following JSON format:
{
    "value": "number",
    "unit": "unit"
}
5. If no clear match is found, assign the most relevant value and unit based on your analysis.
6. Avoid giving None as output.

</SYS>
User: {{ input_str }}
You:"""


class SimpleQA(Component):
    def __init__(self, model_client: ModelClient, model_kwargs: dict):
        super().__init__()
        self.generator = Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=qa_template,
        )

    def call(self, input: dict) -> str:
        return self.generator.call({"input_str": str(input)})

    async def acall(self, input: dict) -> str:
        return await self.generator.acall({"input_str": str(input)})


In [13]:
from lightrag.components.model_client import OllamaClient
from IPython.display import Markdown, display
model = {
    "model_client": OllamaClient(),
    "model_kwargs": {"model": "llama3.1:8b"}
}
import pandas as pd
responses = []
qa = SimpleQA(**model)
for index, row in train_50_df.iterrows():
    input_str = f"Entity: {row['entity_name']}, Text: {row['ocr_text']}"
    output = qa({"input_str": input_str})
    print(output.data)
    responses.append(output)


{
    "value": "500",
    "unit": "gram"
}
{
    "value": null,
    "unit": null
}
{
    "value": "200",
    "unit": "milligram"
}
{
    "value": "468",
    "unit": "gram"
}
{
    "value": "1400",
    "unit": "milligram"
}
{
    "value": "1400",
    "unit": "milligram"
}
{
    "value": "1400",
    "unit": "milligram"
}
{
    "value": "1400",
    "unit": "milligram"
}
{
    "value": "1400",
    "unit": "milligram"
}
{
    "value": "1400",
    "unit": "milligram"
}
{
    "value": "30",
    "unit": "kilogram"
}
{
    "value": "45",
    "unit": "kilogram"
}
{
    "value": "100",
    "unit": "gram"
}
{
    "value": "101g",
    "unit": "gram"
}
{
    "value": "100",
    "unit": "gram"
}
{
    "value": "100",
    "unit": "gram"
}
{
    "value": "200",
    "unit": "gram"
}
{
    "value": "200",
    "unit": "gram"
}
{
    "value": "200",
    "unit": "gram"
}
{
    "value": "200",
    "unit": "gram"
}
{
    "value": "23",
    "unit": "centimeter"
}
{
    "value": "22.2",
    "unit": "inches"
}
{

In [14]:
train_50_df['llm_response'] = responses
print(train_50_df.head())

                                          image_link  group_id  entity_name  \
0  https://m.media-amazon.com/images/I/61I9XdN6OF...    748919  item_weight   
1  https://m.media-amazon.com/images/I/71gSRbyXmo...    916768  item_volume   
2  https://m.media-amazon.com/images/I/61BZ4zrjZX...    459516  item_weight   
3  https://m.media-amazon.com/images/I/612mrlqiI4...    459516  item_weight   
4  https://m.media-amazon.com/images/I/617Tl40LOX...    731432  item_weight   

     entity_value                                           ocr_text  \
0      500.0 gram  PROPOS NATURE INGREDIENT MENAGER MULTI-USAGE T...   
1         1.0 cup  TEARRIFIC LEBENSMITTELECHT M GEPRAGTES DESIGN ...   
2      0.709 gram  COMPOSITION Serving Size:1 Tablet 0.709 g)Each...   
3      0.709 gram  therngiing 460trNS1202NS468id 1201ning  NS470 ...   
4  1400 milligram  Horbaach HIGHSTRENGTH PSYLLIOM HUSK 1400 PLANT...   

                                        llm_response  
0  GeneratorOutput(data='{\n    "valu

In [15]:
train_50_df['llm_response'][0]

GeneratorOutput(data='{\n    "value": "500",\n    "unit": "gram"\n}', error=None, usage=None, raw_response='{\n    "value": "500",\n    "unit": "gram"\n}', metadata=None)

In [16]:
import json

def extract_value_and_unit(generator_output):
    if hasattr(generator_output, 'raw_response') and isinstance(generator_output.raw_response, str):
        try:
            parsed_response = json.loads(generator_output.raw_response)
            value = parsed_response.get("value", "")
            unit = parsed_response.get("unit", "")
            return f"{value} {unit}"
        except json.JSONDecodeError:
            return ""
    return ""


In [17]:
import pandas as pd

train_50_df['cleaned_response'] = train_50_df['llm_response'].apply(extract_value_and_unit)
print(train_50_df[['llm_response', 'cleaned_response']].head())


                                        llm_response cleaned_response
0  GeneratorOutput(data='{\n    "value": "500",\n...         500 gram
1  GeneratorOutput(data='{\n    "value": null,\n ...        None None
2  GeneratorOutput(data='{\n    "value": "200",\n...    200 milligram
3  GeneratorOutput(data='{\n    "value": "468",\n...         468 gram
4  GeneratorOutput(data='{\n    "value": "1400",\...   1400 milligram


In [18]:
train_50_df['cleaned_response']

,cleaned_response
0,500 gram
1,None None
2,200 milligram
3,468 gram
4,1400 milligram
5,1400 milligram
6,1400 milligram
7,1400 milligram
8,1400 milligram
9,1400 milligram


In [19]:
train_df['entity_value'].head(10)

,entity_value
0,500.0 gram
1,1.0 cup
2,0.709 gram
3,0.709 gram
4,1400 milligram
5,1400 milligram
6,1400 milligram
7,1400 milligram
8,1400 milligram
9,1400 milligram


In [20]:
train_df_50 = train_df.head(50)


In [21]:
true_values = train_df_50['entity_value']

predicted_values = train_50_df['cleaned_response']


In [22]:
from sklearn.metrics import f1_score
import numpy as np

true_values_list = true_values.astype(str).tolist()
predicted_values_list = predicted_values.astype(str).tolist()

f1 = f1_score(true_values_list, predicted_values_list, average='weighted')

print(f"F1 Score: {f1}")

### Would be higher considering Amazon Evaluation Criteria

F1 Score: 0.3583333333333333


In [24]:
train_50_df['ocr_text'][2]

'COMPOSITION Serving Size:1 Tablet 0.709 g)Each serving contains (Approx.Values Ingredient Qty./Serving %RDA" *PHOSPHOcomplexSilybin (Sillybum marianum) 200 mg ** Dandelion Taraxacum officinale leaf extract-101 100 mg ** Kutki Picrorhiza kurroa)rhizome extract-0.5%Bitters 50 mg ** Kasani (Cichorium intybus) seed extract-1%Bitters 25 mg ** Punarnava Boerhavia diffusa root extract-0.07%alkaloids 25 mg ** Bhui amla Phyllanthus amarus) WP extract-0.5%Bitters 25 mg Amla (Phyllanthus emblica fruit extract-10%Tannins 25 mg Licorice (Glycyrrhiza glabra) root extract-5% Glycyrrhizin 25 mg Vitamin E 10 mg 100 Piper nigrum fruit extract-95%Piperine 5 mg ** NUTRITIONAL INFORMATION PER SERVING (APPROX.VALUES) Nutrients Qty./Serving %RDA# Energy 3.04kcal 0.13 Carbohydrate 0.51 g ** (Sugars) 0.2g ** Protein 0.04 g 0.07 Fat 0.09 g ** %RDA values established as per ICMR 2010 for sedentarylifestyle-Men **%RDA not established by ICMR'

In [25]:
train_df_first_50 = train_df.head(50).reset_index(drop=True)

comparison_df = pd.DataFrame({
    'Entity_Value (train_df)': train_df_first_50['entity_value'],
    'Cleaned_Response (train_50_df)': train_50_df['cleaned_response']
})
print(comparison_df)



       Entity_Value (train_df) Cleaned_Response (train_50_df)
0                   500.0 gram                       500 gram
1                      1.0 cup                      None None
2                   0.709 gram                  200 milligram
3                   0.709 gram                       468 gram
4               1400 milligram                 1400 milligram
5               1400 milligram                 1400 milligram
6               1400 milligram                 1400 milligram
7               1400 milligram                 1400 milligram
8               1400 milligram                 1400 milligram
9               1400 milligram                 1400 milligram
10               30.0 kilogram                    30 kilogram
11  10 kilogram to 15 kilogram                    45 kilogram
12                  3.53 ounce                       100 gram
13                  3.53 ounce                      101g gram
14                    53 ounce                       100 gram
15      